In [7]:
import pandas as pd

# Loading Datasets
eda = pd.read_csv("../data/processed/eda_shipping_features.csv")
preds = pd.read_csv("../data/processed/model_predictions_inventory.csv")
forecast = pd.read_csv("../data/processed/nike_inventory_forecast.csv")

forecast.rename(columns={"ds": "date"}, inplace=True)

# Merging Datasets
merged = (
    eda.merge(preds, on="date", how="outer")
    .merge(forecast, on="date", how="outer")
    .sort_values("date")
)

# Export dataset for Tableau
merged.to_csv("../data/processed/nike_supplychain_tableau_final.csv", index=False)
print("Tableau dataset saved")
print("Merged shape: ", merged.shape)

Tableau dataset saved
Merged shape:  (129, 14)


In [3]:
import pandas as pd

# Loading cleaned dataset
df = pd.read_csv("../data/processed/nike_supplychain_tableau_final.csv")

# Selecting relevent columns
cols = ["freight_cost_index", "oil_price_usd", "port_congestion_index"]

# Compute correlation
corr = df[cols].corr()

# Saving as csv
corr.to_csv("../data/processed/nike_correlation_matrix.csv")

corr

,freight_cost_index,oil_price_usd,port_congestion_index
freight_cost_index,1.000000,-0.605221,0.639670
oil_price_usd,-0.605221,1.000000,-0.592496
port_congestion_index,0.639670,-0.592496,1.000000
